# Raytracing a polygon mesh

In [1]:
using Krang
import GLMakie as GLMk
using GeometryBasics
using FileIO

metric = Krang.Kerr(0.99) # Kerr metric with a spin of 0.99
θo = 90/180*π # Inclination angle of the observer
ρmax = 12.0 # Horizontal and Vertical extent of the screen
sze = 100 # Resolution of the screen is sze x sze

camera = Krang.SlowLightIntensityCamera(metric, θo, -ρmax, ρmax, -ρmax, ρmax, sze)

Krang.SlowLightIntensityCamera{Float64, Matrix{Krang.SlowLightIntensityPixel{Float64}}}(Krang.Kerr{Float64}(1.0, 0.99), Krang.SlowLightIntensityScreen{Float64, Matrix{Krang.SlowLightIntensityPixel{Float64}}}((-12.0, 12.0), (-12.0, 12.0), Krang.SlowLightIntensityPixel{Float64}[Krang.SlowLightIntensityPixel{Float64}(Krang.Kerr{Float64}(1.0, 0.99), (-12.0, -12.0), (-17.811393086775468 - 3.6622313808008166e-17im, 0.3222351305493678 + 1.2554506945771609e-15im, 1.5419781346644301 - 1.2759301128716357e-15im, 15.947179821561672 + 5.7101732102483e-17im), 0.10027124075812203, 3.1948265198800945, 7.300113474580925, -2.1933076951002244, -13.17634564256269, -0.2458426263266534, -0.14259682643912724, (0.0, 0.18519854383797518), (5.102694996447269e-18, 0.2621141790545372), (1.4968675014460734e-15, 0.04636852782576966), 1.5707963267948966, 144.0, 12.0) Krang.SlowLightIntensityPixel{Float64}(Krang.Kerr{Float64}(1.0, 0.99), (-12.0, -11.757575757575758), (-17.638476027306 - 2.5119566969130612e-17im, 0.31

Import external mesh

In [2]:
bunny_mesh = translate(
    rotate(
        scale(
            load(download("https://graphics.stanford.edu/~mdfisher/Data/Meshes/bunny.obj", "bunny.obj")), 100
        ),π/2, 1.0, 0.0, 0.0
    ), 2.0,7.0,-10.0
);

Raytrace the mesh embedded in the Kerr space time. The emission picked up by a ray will be the sum of all the intersections of the ray with the mesh
The mesh is embedded in the space-time by assuming the world coordinates of the mesh is the same as the Boyer-Lindquist coordinates of the space time.
The ray tracing is done by scanning over the image one line at a time

Let us now raytrace the image to see what the mesh looks like from the observer's perspective.

In [3]:
intersections = raytrace(camera, bunny_mesh);

And plot the image with GLMakie,

In [4]:
fig = GLMk.Figure()
ax = GLMk.Axis(fig[1,1], aspect=1)
GLMk.heatmap!(ax, intersections, colormap=:afmhot);

save("mesh_geometry_example.png", fig);

![image](mesh_geometry_example.png)

Below is a short plotting routine that generates a video showing the scene from three different perspectives, and live renders the image.

In [5]:
fig = GLMk.Figure()
ax1 = GLMk.Axis3(fig[1,1], aspect=(1,1,1), title="Top view of scene", elevation = π/2, azimuth = π)
ax2 = GLMk.Axis3(fig[1,2], aspect=(1,1,1), title="Side view of scene", elevation = 0.0, azimuth = 3π/2)
ax3 = GLMk.Axis3(fig[2,1], aspect=(1,1,1), title="Isometric view of scene")

ax = GLMk.Axis(fig[2,2], aspect=1, title="Heatmap of ray intersections with mesh")

for a in [ax1, ax2, ax3]
    GLMk.xlims!(a, (-10, 10))
    GLMk.ylims!(a, (-10, 10))
    GLMk.zlims!(a, (-10, 10))
    GLMk.hidedecorations!(a)
end

GLMk.hidedecorations!(ax)
sphere = GLMk.Sphere(GLMk.Point(0.0,0.0,0.0), horizon(metric)) # Sphere to represent black hole
lines_to_plot = Krang.generate_ray.(camera.screen.pixels, 100) # 100 is the number of steps to take along the ray

img = zeros(sze, sze)
recording = GLMk.record(fig, "mesh.mp4", 1:sze*sze, framerate=120) do i
    line = lines_to_plot[i]

    img[i] = intersections[i]

    GLMk.empty!(ax)
    GLMk.empty!(ax1)
    GLMk.empty!(ax2)
    GLMk.empty!(ax3)

    for a in [ax1, ax2, ax3]
        GLMk.mesh!(a, bunny_mesh, color=[parse(GLMk.Colorant, "rgba(0%, 50%, 70%,1.0)") for tri in bunny_mesh.position], colormap = :blues, transparency=true)
        GLMk.mesh!(a, sphere, color=:black)
    end

    GLMk.lines!(ax3, line, color=:red)
    GLMk.heatmap!(ax, img, colormap=:afmhot, colorrange=(0, 8))
end

"mesh.mp4"

<video loop muted playsinline controls src="./mesh.mp4" />

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*